# Data Engineering Capstone Project - FIFA Players
### Data Engineering Capstone Project

#### Project Summary
Apply concepts and technologies learned during the Data Engineering Nanodegree program.

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [277]:
# Do all imports and installs here
import pandas as pd
import numpy as np

# cd /Users/<user_name>/opt/anaconda3/condabin
# ./conda install pyspark
import pyspark
from pyspark.sql.functions import *

### Step 1: Scope the Project and Gather Data

#### Scope 
The goal of the work will be to present a cloud based Data Warehouse, that allows the user to analyse and response questions about the performance of soccer teams and players based on info from sport history and EA Sports videogame.
In order to familiarize myself with other cloud solutions alternatives, some technologies used will be from Azure Cloud.

#### Datasets
The data that we are going to work on comes from three differents sources:

* FIFA 20 complete player dataset (https://www.kaggle.com/stefanoleone992/fifa-20-complete-player-dataset)

    - The dataset is compose of six csv files (last 6 versions of the videogame)
    - Player positions, with the role in the club and in the national team
    - Player attributes with statistics as Attacking, Skills, Defense, Mentality, GK Skills, etc.
    - Player personal data like Nationality, Club, DateOfBirth, Wage, Salary, etc.

* The Big Five European soccer leagues data (https://www.kaggle.com/hikne707/big-five-european-soccer-leagues)

    - All game scores of the big five European soccer leagues (England, Germany, Spain, Italy and France) for the 1995/96 to 2019/20 seasons.

In [278]:
# Read in the data here
df_2015 = pd.read_csv('sources/fifa-20-complete-player-dataset/players_15.csv')

In [279]:
df_2015.head()

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,work_rate,body_type,real_face,release_clause_eur,player_tags,team_position,team_jersey_number,loaned_from,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning,player_traits,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,...,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,27,1987-06-24,169,67,Argentina,FC Barcelona,93,95,0,0,CF,Left,5,3,4,Medium/Low,Normal,Yes,NaN,"#Speedster, #Dribbler, #FK Specialist, #Acroba...",CF,10.0,NaN,2004-07-01,2018.0,CF,10.0,93.0,89.0,86.0,96.0,27.0,63.0,NaN,NaN,NaN,NaN,NaN,NaN,"Finesse Shot, Speed Dribbler (CPU AI Only), On...",84,94-3,71,89,85-5,96-1,...,96,90+2,94,94-2,95,80-1,73,77-1,60,88,48,22,92-1,90,76,NaN,25,21,20,6,11,15,14,8,89+3,89+3,89+3,92+3,90+3,90+3,90+3,92+3,92+3,92+3,92+3,90+3,79+3,79+3,79+3,90+3,62+3,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,29,1985-02-05,185,80,Portugal,Real Madrid,92,92,0,0,"LW, LM",Right,5,4,5,High/Low,Normal,Yes,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...",LW,7.0,NaN,2009-07-01,2018.0,LW,7.0,93.0,93.0,81.0,91.0,32.0,79.0,NaN,NaN,NaN,NaN,NaN,NaN,"Power Free-Kick, Flair, Long Shot Taker (CPU A...",83,95+3,86,82,87+2,93+3,...,91,94,93,90,63-12,94,94,89,79,93,63,24,91+2,81,85,NaN,22,31,23,7,11,15,14,11,91+3,91+3,91+3,89+3,91+3,91+3,91+3,89+3,89+3,89+3,89+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3
2,9014,https://sofifa.com/player/9014/arjen-robben/15...,A. Robben,Arjen Robben,30,1984-01-23,180,80,Netherlands,FC Bayern München,90,90,0,0,"RM, LM, RW",Left,5,2,4,High/Low,Normal,Yes,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...",SUB,10.0,NaN,2009-08-28,2017.0,RS,11.0,93.0,86.0,83.0,92.0,32.0,64.0,NaN,NaN,NaN,NaN,NaN,NaN,"Diver, Injury Prone, Avoids Using Weaker Foot,...",80,85+2,50,86+2,86+2,93,...,93,93+2,93+1,89+2,91,86,61,78+11,65,90+1,47+9,39+1,89+2,84+2,80+1,NaN,29,26+2,26+1,10,8,11,5,15,84+3,84+3,84+3,88+3,87+3,87+3,87+3,88+3,88+3,88+3,88+3,87+3,78+3,78+3,78+3,87+3,64+3,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3
3,41236,https://sofifa.com/player/41236/zlatan-ibrahim...,Z. Ibrahimović,Zlatan Ibrahimović,32,1981-10-03,195,95,Sweden,Paris Saint-Germain,90,90,0,0,ST,Right,5,4,4,Medium/Low,Normal,Yes,NaN,"#Poacher, #Aerial Threat, #Distance Shooter, #...",ST,10.0,NaN,2012-07-01,2016.0,ST,10.0,76.0,91.0,81.0,86.0,34.0,86.0,NaN,NaN,NaN,NaN,NaN,NaN,"Power Free-Kick, Leadership, Flair, Long Shot ...",76,91,76,84-2,92+3,88,...,74+2,77-1,86,85,41,93,72,78,93,88,84,20,86,83-1,91,NaN,25,41-8,27,13,15,10,9,12,87+3,87+3,87+3,84+3,86+3,86+3,86+3,84+3,86+3,86+3,86+3,83+3,76+3,76+3,76+3,83+3,61+3,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3
4,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Neuer,28,1986-03-27,193,92,Germany,FC Bayern München,90,90,0,0,GK,Right,5,4,1,Medium/Medium,Normal,Yes,NaN,NaN,GK,1.0,NaN,2011-07-01,2019.0,GK,1.0,NaN,NaN,NaN,NaN,NaN,NaN,87.0,85.0,92.0,86.0,60.0,90.

In [280]:
df_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15465 entries, 0 to 15464
Columns: 104 entries, sofifa_id to rb
dtypes: float64(17), int64(11), object(76)
memory usage: 12.3+ MB


In [281]:
list(df_2015.columns)

['sofifa_id',
 'player_url',
 'short_name',
 'long_name',
 'age',
 'dob',
 'height_cm',
 'weight_kg',
 'nationality',
 'club',
 'overall',
 'potential',
 'value_eur',
 'wage_eur',
 'player_positions',
 'preferred_foot',
 'international_reputation',
 'weak_foot',
 'skill_moves',
 'work_rate',
 'body_type',
 'real_face',
 'release_clause_eur',
 'player_tags',
 'team_position',
 'team_jersey_number',
 'loaned_from',
 'joined',
 'contract_valid_until',
 'nation_position',
 'nation_jersey_number',
 'pace',
 'shooting',
 'passing',
 'dribbling',
 'defending',
 'physic',
 'gk_diving',
 'gk_handling',
 'gk_kicking',
 'gk_reflexes',
 'gk_speed',
 'gk_positioning',
 'player_traits',
 'attacking_crossing',
 'attacking_finishing',
 'attacking_heading_accuracy',
 'attacking_short_passing',
 'attacking_volleys',
 'skill_dribbling',
 'skill_curve',
 'skill_fk_accuracy',
 'skill_long_passing',
 'skill_ball_control',
 'movement_acceleration',
 'movement_sprint_speed',
 'movement_agility',
 'movement_re

In [282]:
# For this first source (compose of 6 csv files in total), we have 104 columns in total, 
# for our porpuses we selected 73 columns in total

columns_names_selected = ['sofifa_id', 'player_url', 'short_name', 'long_name',
            'age', 'dob', 'height_cm', 'weight_kg',
            'nationality', 'club', 'overall', 'potential',
            'value_eur','wage_eur', 'player_positions',
            'preferred_foot', 'international_reputation', 'weak_foot',
            'skill_moves', 'release_clause_eur', 'player_tags', 'team_position',
            'team_jersey_number', 'joined', 'contract_valid_until',
            'nation_position', 'nation_jersey_number', 'pace', 'shooting', 'passing',
            'dribbling', 'defending', 'physic', 'gk_diving', 'gk_handling', 'gk_kicking',
            'gk_reflexes', 'gk_speed', 'gk_positioning', 'attacking_crossing',
            'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing',
            'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy',
            'skill_long_passing', 'skill_ball_control', 'movement_acceleration',
            'movement_sprint_speed', 'movement_agility', 'movement_reactions',
            'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina',
            'power_strength', 'power_long_shots', 'mentality_aggression',
            'mentality_interceptions', 'mentality_positioning', 'mentality_vision',
            'mentality_penalties', 'mentality_composure', 'defending_marking',
            'defending_standing_tackle', 'defending_sliding_tackle', 'goalkeeping_diving',
            'goalkeeping_handling', 'goalkeeping_kicking', 'goalkeeping_positioning',
            'goalkeeping_reflexes']

df_2015 = df_2015[columns_names_selected]
df_2015.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15465 entries, 0 to 15464
Data columns (total 73 columns):
sofifa_id                     15465 non-null int64
player_url                    15465 non-null object
short_name                    15465 non-null object
long_name                     15465 non-null object
age                           15465 non-null int64
dob                           15465 non-null object
height_cm                     15465 non-null int64
weight_kg                     15465 non-null int64
nationality                   15465 non-null object
club                          15465 non-null object
overall                       15465 non-null int64
potential                     15465 non-null int64
value_eur                     15465 non-null int64
wage_eur                      15465 non-null int64
player_positions              15465 non-null object
preferred_foot                15465 non-null object
international_reputation      15465 non-null int64
weak_foot       

In [283]:
# we made the same selection for the other five csv files

df_2016 = pd.read_csv('sources/fifa-20-complete-player-dataset/players_16.csv')
df_2017 = pd.read_csv('sources/fifa-20-complete-player-dataset/players_17.csv')
df_2018 = pd.read_csv('sources/fifa-20-complete-player-dataset/players_18.csv')
df_2019 = pd.read_csv('sources/fifa-20-complete-player-dataset/players_19.csv')
df_2020 = pd.read_csv('sources/fifa-20-complete-player-dataset/players_20.csv')

df_2016 = df_2016[columns_names_selected]
df_2017 = df_2017[columns_names_selected]
df_2018 = df_2018[columns_names_selected]
df_2019 = df_2019[columns_names_selected]
df_2020 = df_2020[columns_names_selected]

In [284]:
# In order to manage just one dataframe for this source, we concatenate the 6 dataframes

frames = [df_2015, df_2016, df_2017, df_2018, df_2019, df_2020]
df_fifa = pd.concat(frames)

In [285]:
pd.set_option('display.max_columns', 100)
df_fifa.head(10)

,sofifa_id,player_url,short_name,long_name,age,dob,height_cm,weight_kg,nationality,club,overall,potential,value_eur,wage_eur,player_positions,preferred_foot,international_reputation,weak_foot,skill_moves,release_clause_eur,player_tags,team_position,team_jersey_number,joined,contract_valid_until,nation_position,nation_jersey_number,pace,shooting,passing,dribbling,defending,physic,gk_diving,gk_handling,gk_kicking,gk_reflexes,gk_speed,gk_positioning,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,158023,https://sofifa.com/player/158023/lionel-messi/...,L. Messi,Lionel Andrés Messi Cuccittini,27,1987-06-24,169,67,Argentina,FC Barcelona,93,95,0,0,CF,Left,5,3,4,NaN,"#Speedster, #Dribbler, #FK Specialist, #Acroba...",CF,10.0,2004-07-01,2018.0,CF,10.0,93.0,89.0,86.0,96.0,27.0,63.0,NaN,NaN,NaN,NaN,NaN,NaN,84,94-3,71,89,85-5,96-1,89+2,90+4,76,96,96,90+2,94,94-2,95,80-1,73,77-1,60,88,48,22,92-1,90,76,NaN,25,21,20,6,11,15,14,8
1,20801,https://sofifa.com/player/20801/c-ronaldo-dos-...,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,29,1985-02-05,185,80,Portugal,Real Madrid,92,92,0,0,"LW, LM",Right,5,4,5,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...",LW,7.0,2009-07-01,2018.0,LW,7.0,93.0,93.0,81.0,91.0,32.0,79.0,NaN,NaN,NaN,NaN,NaN,NaN,83,95+3,86,82,87+2,93+3,88,79,72,92-3,91,94,93,90,63-12,94,94,89,79,93,63,24,91+2,81,85,NaN,22,31,23,7,11,15,14,11
2,9014,https://sofifa.com/player/9014/arjen-robben/15...,A. Robben,Arjen Robben,30,1984-01-23,180,80,Netherlands,FC Bayern München,90,90,0,0,"RM, LM, RW",Left,5,2,4,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...",SUB,10.0,2009-08-28,2017.0,RS,11.0,93.0,86.0,83.0,92.0,32.0,64.0,NaN,NaN,NaN,NaN,NaN,NaN,80,85+2,50,86+2,86+2,93,85,83-1,76+3,90+1,93,93+2,93+1,89+2,91,86,61,78+11,65,90+1,47+9,39+1,89+2,84+2,80+1,NaN,29,26+2,26+1,10,8,11,5,15
3,41236,https://sofifa.com/player/41236/zlatan-ibrahim...,Z. Ibrahimović,Zlatan Ibrahimović,32,1981-10-03,195,95,Sweden,Paris Saint-Germain,90,90,0,0,ST,Right,5,4,4,NaN,"#Poacher, #Aerial Threat, #Distance Shooter, #...",ST,10.0,2012-07-01,2016.0,ST,10.0,76.0,91.0,81.0,86.0,34.0,86.0,NaN,NaN,NaN,NaN,NaN,NaN,76,91,76,84-2,92+3,88,80,80,76+4,90,74+2,77-1,86,85,41,93,72,78,93,88,84,20,86,83-1,91,NaN,25,41-8,27,13,15,10,9,12
4,167495,https://sofifa.com/player/167495/manuel-neuer/...,M. Neuer,Manuel Neuer,28,1986-03-27,193,92,Germany,FC Bayern München,90,90,0,0,GK,Right,5,4,1,NaN,NaN,GK,1.0,2011-07-01,2019.0,GK,1.0,NaN,NaN,NaN,NaN,NaN,NaN,87.0,85.0,92.0,86.0,60.0,90.0,25,25,25,42,25,25,25,25,41,31,58+8,61+7,43+4,89+3,35,42,78,44+9,83+5,25,29+3,30+8,25,20,37,NaN,25,25,25,87,85+3,92,90+6,86
5,176580,https://sofifa.com/player/176580/luis-suarez/1...,L. Suárez,Luis Alberto Suárez Díaz,27,1987-01-24,181,81,Uruguay,FC Barcelona,89,91,0,0,"ST, CF",Right,5,4,4,NaN,"#Acrobat, #Clinical Finisher",RES,9.0,2014-07-11,2019.0,NaN,NaN,83.0,87.0,79.0,88.0,42.0,79.0,NaN,NaN,NaN,NaN,NaN,NaN,77,91+2,75,82,85,90-2,86,84,64,89,88,79,86,91+2,60-25,84,69,86,76,82-2,78,41,88+2,84,85,NaN,30,45,38,27,25,31,33,37
6,183277,https://sofifa.com/player/183277/eden-hazard/1...,E. Hazard,Eden Hazard,23,1991-01-07,173,74,Belgium,Chelsea,88,90,0,0,"LM, RM",Right,4,4,4,NaN,"#Speedster, #Dribbler, #Acrobat",LM,10.0,2012-07-01,2017.0,LM,10.0,90.0,82.0,84.0,91.0,32.0,64.0,NaN,NaN,NaN,NaN,NaN,NaN,78-4,83,57,87+1,79,92+2,82,79,82,89,93+2,87,92,85+1,90,79,59,74+1,63,82,54,41,84,86+1,8

In [286]:
# Save locally the concatenated dataframe
df_fifa.to_csv('sources/fifa-20-complete-player-dataset/fifa_players.csv')

In [287]:
# Load of the European soccer leagues source
df_euro_league = pd.read_csv('sources/BIG FIVE 1995-2019.csv')

In [288]:
df_euro_league.head()

,Round,Date,Team 1,FT,HT,Team 2,Year,Country,FT Team 1,FT Team 2,HT Team 1,HT Team 2,GGD,Team 1 (pts),Team 2 (pts)
0,1,(Sat) 19 Aug 1995 (W33),Aston Villa FC,3-1,3-0,Manchester United FC,1995,ENG,3,1,3,0,2,3,0
1,1,(Sat) 19 Aug 1995 (W33),Blackburn Rovers FC,1-0,1-0,Queens Park Rangers FC,1995,ENG,1,0,1,0,1,3,0
2,1,(Sat) 19 Aug 1995 (W33),Chelsea FC,0-0,0-0,Everton FC,1995,ENG,0,0,0,0,0,1,1
3,1,(Sat) 19 Aug 1995 (W33),Liverpool FC,1-0,0-0,Sheffield Wednesday FC,1995,ENG,1,0,0,0,1,3,0
4,1,(Sat) 19 Aug 1995 (W33),Manchester City FC,1-1,0-1,Tottenham Hotspur FC,1995,ENG,1,1,0,1,0,1,1


In [289]:
# Compare teams names between the two sources
df_fifa_teams = df_fifa['club'].unique()
df_fifa_teams = pd.DataFrame(df_fifa_teams)
df_fifa_teams.head(n=20)

,0
0,FC Barcelona
1,Real Madrid
2,FC Bayern München
3,Paris Saint-Germain
4,Chelsea
5,Manchester United
6,Manchester City
7,Borussia Dortmund
8,Arsenal
9,Juventus


In [290]:
df_teams_euro = df_euro_league['Team 1'].unique()
df_teams_euro = pd.DataFrame(df_teams_euro)
df_teams_euro.head(n=20)

,0
0,Aston Villa FC
1,Blackburn Rovers FC
2,Chelsea FC
3,Liverpool FC
4,Manchester City FC
5,Newcastle United FC
6,Southampton FC
7,West Ham United FC
8,Wimbledon FC
9,Arsenal FC


In [291]:
from pyspark.sql import SparkSession
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

spark_df = spark.read.csv('sources/fifa-20-complete-player-dataset/fifa_players.csv', header='true')
type(spark_df)

pyspark.sql.dataframe.DataFrame

In the next step we will focus on standardize the date format, as well the team names

### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### Cleaning Steps
Document steps necessary to clean the data

In the prevous step we selected the columns that are relevant for our porpuses.
Now we will start checking the missing values.

In [292]:
#missing values for fifa players data source
for column in df_fifa:
    if df_fifa[column].isnull().any():
        print('{0} has {1} null values'.format(column, df_fifa[column].isnull().sum()))

release_clause_eur has 51644 null values
player_tags has 92698 null values
team_position has 1326 null values
team_jersey_number has 1326 null values
joined has 8038 null values
contract_valid_until has 1333 null values
nation_position has 94470 null values
nation_jersey_number has 94470 null values
pace has 11247 null values
shooting has 11247 null values
passing has 11247 null values
dribbling has 11247 null values
defending has 11247 null values
physic has 11247 null values
gk_diving has 89748 null values
gk_handling has 89748 null values
gk_kicking has 89748 null values
gk_reflexes has 89748 null values
gk_speed has 89748 null values
gk_positioning has 89748 null values
mentality_composure has 30346 null values


Looking the data and the fields involved in null values, we realized that there is no need to replace values, normally are indicator that differences a player and a goalkeeper.

In [293]:
#missing values for euro league data source
total=0
for column in df_euro_league:
    if df_euro_league[column].isnull().any():
        total+=1
        print('{0} has {1} null values'.format(column, df_euro_league[column].isnull().sum()))
if total==0:
    print('No null values for euro league data source.')

No null values for euro league data source.


In [294]:
# Check for dupolicate data for fifa players data source
df1 = spark_df.groupBy(columns_names_selected).count().filter("count > 1")
df1.show()

+---------+----------+----------+---------+---+---+---------+---------+-----------+----+-------+---------+---------+--------+----------------+--------------+------------------------+---------+-----------+------------------+-----------+-------------+------------------+------+--------------------+---------------+--------------------+----+--------+-------+---------+---------+------+---------+-----------+----------+-----------+--------+--------------+------------------+-------------------+--------------------------+-----------------------+-----------------+---------------+-----------+-----------------+------------------+------------------+---------------------+---------------------+----------------+------------------+----------------+----------------+-------------+-------------+--------------+----------------+--------------------+-----------------------+---------------------+----------------+-------------------+-------------------+-----------------+-------------------------+---------------

In [295]:
# Check for dupolicate data for euro league data source
columns_names = ['Round', 'Date', 'Team 1', 'FT', 'HT', 'Team 2', 'Year', 'Country', 'FT Team 1', 
                 'FT Team 2', 'HT Team 1', 'HT Team 2', 'GGD', 'Team 1 (pts)', 'Team 2 (pts)']
spark_df_euro = spark.read.csv('sources/BIG FIVE 1995-2019.csv', header='true')
df2 = spark_df_euro.groupBy(columns_names).count().filter("count > 1")
df2.show()

+-----+----+------+---+---+------+----+-------+---------+---------+---------+---------+---+------------+------------+-----+
|Round|Date|Team 1| FT| HT|Team 2|Year|Country|FT Team 1|FT Team 2|HT Team 1|HT Team 2|GGD|Team 1 (pts)|Team 2 (pts)|count|
+-----+----+------+---+---+------+----+-------+---------+---------+---------+---------+---+------------+------------+-----+
+-----+----+------+---+---+------+----+-------+---------+---------+---------+---------+---+------------+------------+-----+



No duplicate data found for the sources

In [296]:
# Performing cleaning tasks on euro league data source

#Getting info
spark_df_euro = spark.read.csv('sources/BIG FIVE 1995-2019.csv', header='true')
spark_df_euro.createOrReplaceTempView("EURO_LEAGUE")

query = 'SELECT * FROM EURO_LEAGUE'
spark_df = spark.sql(query)
spark_df.limit(20).toPandas().head(n=20)

,Round,Date,Team 1,FT,HT,Team 2,Year,Country,FT Team 1,FT Team 2,HT Team 1,HT Team 2,GGD,Team 1 (pts),Team 2 (pts)
0,1,(Sat) 19 Aug 1995 (W33),Aston Villa FC,3-1,3-0,Manchester United FC,1995,ENG,3,1,3,0,2,3,0
1,1,(Sat) 19 Aug 1995 (W33),Blackburn Rovers FC,1-0,1-0,Queens Park Rangers FC,1995,ENG,1,0,1,0,1,3,0
2,1,(Sat) 19 Aug 1995 (W33),Chelsea FC,0-0,0-0,Everton FC,1995,ENG,0,0,0,0,0,1,1
3,1,(Sat) 19 Aug 1995 (W33),Liverpool FC,1-0,0-0,Sheffield Wednesday FC,1995,ENG,1,0,0,0,1,3,0
4,1,(Sat) 19 Aug 1995 (W33),Manchester City FC,1-1,0-1,Tottenham Hotspur FC,1995,ENG,1,1,0,1,0,1,1
5,1,(Sat) 19 Aug 1995 (W33),Newcastle United FC,3-0,1-0,Coventry City FC,1995,ENG,3,0,1,0,3,3,0
6,1,(Sat) 19 Aug 1995 (W33),Southampton FC,3-4,1-3,Nottingham Forest FC,1995,ENG,3,4,1,3,1,0,3
7,1,(Sat) 19 Aug 1995 (W33),West Ham United FC,1-2,1-0,Leeds United FC,1995,ENG,1,2,1,0,1,0,3
8,1,(Sat) 19 Aug 1995 (W33),Wimbledon FC,3-2,2-2,Bolton Wanderers FC,1995,ENG,3,2,2,2,1,3,0
9,1,(Sun) 20 Aug 1995 (W33),Arsenal FC,1-1,1-1,Middlesbrough FC,1995,ENG,1,1,1,1,0,1,1


In [297]:
#Remove the text ' FC' at the end of team names
newDf = spark_df.withColumn('Team 1', regexp_replace('Team 1', ' FC', ''))
newDf = newDf.withColumn('Team 2', regexp_replace('Team 2', ' FC', ''))
newDf.limit(20).toPandas().head(n=20)

,Round,Date,Team 1,FT,HT,Team 2,Year,Country,FT Team 1,FT Team 2,HT Team 1,HT Team 2,GGD,Team 1 (pts),Team 2 (pts)
0,1,(Sat) 19 Aug 1995 (W33),Aston Villa,3-1,3-0,Manchester United,1995,ENG,3,1,3,0,2,3,0
1,1,(Sat) 19 Aug 1995 (W33),Blackburn Rovers,1-0,1-0,Queens Park Rangers,1995,ENG,1,0,1,0,1,3,0
2,1,(Sat) 19 Aug 1995 (W33),Chelsea,0-0,0-0,Everton,1995,ENG,0,0,0,0,0,1,1
3,1,(Sat) 19 Aug 1995 (W33),Liverpool,1-0,0-0,Sheffield Wednesday,1995,ENG,1,0,0,0,1,3,0
4,1,(Sat) 19 Aug 1995 (W33),Manchester City,1-1,0-1,Tottenham Hotspur,1995,ENG,1,1,0,1,0,1,1
5,1,(Sat) 19 Aug 1995 (W33),Newcastle United,3-0,1-0,Coventry City,1995,ENG,3,0,1,0,3,3,0
6,1,(Sat) 19 Aug 1995 (W33),Southampton,3-4,1-3,Nottingham Forest,1995,ENG,3,4,1,3,1,0,3
7,1,(Sat) 19 Aug 1995 (W33),West Ham United,1-2,1-0,Leeds United,1995,ENG,1,2,1,0,1,0,3
8,1,(Sat) 19 Aug 1995 (W33),Wimbledon,3-2,2-2,Bolton Wanderers,1995,ENG,3,2,2,2,1,3,0
9,1,(Sun) 20 Aug 1995 (W33),Arsenal,1-1,1-1,Middlesbrough,1995,ENG,1,1,1,1,0,1,1


In [298]:
#Cleaning date field
newDf = newDf.withColumn("New_Date", lpad(concat(split(newDf['Date'], ' ')[1], 
                                                 lit(' '), split(newDf['Date'], ' ')[2], lit(' '), 
                                                 split(newDf['Date'], ' ')[3]), 11, '0'))

newDf.limit(20).toPandas().head(n=20)

,Round,Date,Team 1,FT,HT,Team 2,Year,Country,FT Team 1,FT Team 2,HT Team 1,HT Team 2,GGD,Team 1 (pts),Team 2 (pts),New_Date
0,1,(Sat) 19 Aug 1995 (W33),Aston Villa,3-1,3-0,Manchester United,1995,ENG,3,1,3,0,2,3,0,19 Aug 1995
1,1,(Sat) 19 Aug 1995 (W33),Blackburn Rovers,1-0,1-0,Queens Park Rangers,1995,ENG,1,0,1,0,1,3,0,19 Aug 1995
2,1,(Sat) 19 Aug 1995 (W33),Chelsea,0-0,0-0,Everton,1995,ENG,0,0,0,0,0,1,1,19 Aug 1995
3,1,(Sat) 19 Aug 1995 (W33),Liverpool,1-0,0-0,Sheffield Wednesday,1995,ENG,1,0,0,0,1,3,0,19 Aug 1995
4,1,(Sat) 19 Aug 1995 (W33),Manchester City,1-1,0-1,Tottenham Hotspur,1995,ENG,1,1,0,1,0,1,1,19 Aug 1995
5,1,(Sat) 19 Aug 1995 (W33),Newcastle United,3-0,1-0,Coventry City,1995,ENG,3,0,1,0,3,3,0,19 Aug 1995
6,1,(Sat) 19 Aug 1995 (W33),Southampton,3-4,1-3,Nottingham Forest,1995,ENG,3,4,1,3,1,0,3,19 Aug 1995
7,1,(Sat) 19 Aug 1995 (W33),West Ham United,1-2,1-0,Leeds United,1995,ENG,1,2,1,0,1,0,3,19 Aug 1995
8,1,(Sat) 19 Aug 1995 (W33),Wimbledon,3-2,2-2,Bolton Wanderers,1995,ENG,3,2,2,2,1,3,0,19 Aug 1995
9,1,(Sun) 20 Aug 1995 (W33),Arsenal,1-1,1-1,Middlesbrough,1995,ENG,1,1,1,1,0,1,1,20 Aug 1995


In [300]:
#save dataframe cleaned
newDf.write.csv('sources/clean_BIG_FIVE_1995-2019.csv')

### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model

![title](img/Data_Model.png)

#### 3.2 Mapping Out Data Pipelines

- Create an Azure Blob Storage
- Copy cleaned source files to Blob Storage
- Create an Azure Data Factory
- Create a SQL Database
- Use the SQL query file (create_tables.sql) in order to create the tables database.
- Using Data Factory, implement the ETL process from the files to Azure SQL Datawarehouse

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [ ]:
# Write code here

#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.